In [1]:
import math
import pandas as pd

In [ ]:
def calculate_quadrupole_quadrupole_energies():
    

In [24]:
def calculate_monopole_monopole_energies(charge_list,charge):
    energy = 0.0
    for i in range(0,len(charge_list)-1):
        for j in range(i+1,len(charge_list)):
            energy = energy + cal_electrostatic_energy(charge_list[i],charge_list[j],charge[i],charge[j])
    return energy

In [25]:
calculate_monopole_monopole_energies([[1,2,3],[1,3,4],[0,0,0]],[-1,2,-1])

-3.9501694904058985e-28

In [3]:
def calculate_dipole_energies(anion,cation,charge_n,charge_p):
    energy = 0.0
    for i in range(0,len(anion)):
        q_n = charge_n[i]
        for j in range(0,len(cation)):
            q_p = charge_p[j]
            e_new =  cal_electrostatic_energy(anion[i],cation[j],q_n,q_p)
#             print(j)
            energy = energy + e_new
    return energy

In [27]:
def cal_electrostatic_energy(point1,point2,q_n,q_p):
#     print("test "+str(point1))
#     print(q_p)
#     print(q_n)
    dis = math.sqrt(math.pow((point1[0]-point2[0]),2) + math.pow((point1[1]-point2[1]),2)+ math.pow((point1[2]-point2[2]),2))
#     print(dis)
    if (dis == 0):  #for cases where two dipoles with common charge is present.
        return 0
    else:
        #convert angstorm to meters bcz bond distances are in angstorm unit.
        dis = dis*math.pow(10,-10)
        #convert partial charge to elementary charge unit
        ecu = math.pow(1.602*math.pow(10,-19),2)
        energy = (q_p * q_n * ecu )/dis
#         print("distance" + str(dis))
#         print("coulomb energy "+str(energy))
        return energy

In [5]:
def calculate_dipole_dipole_energies(anion,cation,charge_n,charge_p):
    list_of_dipoles = []
    dipole_dipole_energy = 0
    energy = 0
    for i in range(0,len(cation)):
        for j in range(0,len(anion)):
            list_of_dipoles.append((cation[i],anion[j],charge_p[i],charge_n[j]))
#     print(list_of_dipoles)
    for i in range(0,len(list_of_dipoles)-1):
        dpA_cation = list_of_dipoles[i][0]
        dpA_anion = list_of_dipoles[i][1]
        dpA_charge_p = list_of_dipoles[i][2]
        dpA_charge_n = list_of_dipoles[i][3]
#         print(list_of_dipoles[i])
        for j in range(1,len(list_of_dipoles)):
            dpB_cation = list_of_dipoles[j][0]
            dpB_anion = list_of_dipoles[j][1]
            dpB_charge_p = list_of_dipoles[j][2]
            dpB_charge_n = list_of_dipoles[j][3]
            energy = energy + cal_electrostatic_energy(dpA_cation,dpB_cation,dpA_charge_p,dpB_charge_p)
            energy = energy + cal_electrostatic_energy(dpA_cation,dpB_anion,dpA_charge_p,dpB_charge_n)
            energy = energy + cal_electrostatic_energy(dpA_anion,dpB_cation,dpA_charge_n,dpB_charge_p)
            energy = energy + cal_electrostatic_energy(dpA_anion,dpB_anion,dpA_charge_n,dpB_charge_n)
    dipole_dipole_energy = energy
    return dipole_dipole_energy

### create atom_file [contains coordinates and their partial charges]

In [6]:
filepath = "AA_files/"
filename = "molecule_file_Chain_C.xlsx"
molecule_data = pd.read_excel(filepath+filename,engine="openpyxl")    

In [7]:
molecule_data[0:3]

,Atom_number,Residue_name,Chain_name,Atom_name,x,y,z,charge,mass
0,1,1U,C,O5',4.624,2.259,3.276,-0.66,15.9994
1,2,1U,C,H5T,4.586,2.282,3.364,0.43,1.0080
2,3,1U,C,C5',4.690,2.375,3.225,0.05,12.0110


#### create a dictionary containing every residue and its atoms coordinates and charges


In [8]:
residue_name_list = []
residue_dict = {}
for residue in molecule_data['Residue_name']:
    residue_name_list.append(residue)
residue_dict = dict.fromkeys(residue_name_list) 
for key in residue_dict.keys():
    residue_dict[key] = {'x':[],'y':[],'z':[],'charge':[],'mass':[]}

#### For every residue, get the atoms and their coords, mass and charges.


In [9]:
for item,row in molecule_data.iterrows():
    residue_dict[row['Residue_name']]['x'].append(row['x'])
    residue_dict[row['Residue_name']]['y'].append(row['y'])
    residue_dict[row['Residue_name']]['z'].append(row['z'])
    residue_dict[row['Residue_name']]['charge'].append(row['charge'])
    residue_dict[row['Residue_name']]['mass'].append(row['mass'])

#### for each residue get the list of anions and cations

In [10]:
#check partial charge of each atom in the residue and divide into cations and anions
residue_name_list2 = []
[ residue_name_list2.append(x) for x in residue_name_list if x not in residue_name_list2  ]
cation = []
anion = []
charge_n = []
charge_p = []
res_len = residue_name_list2
dipole_energy = 0.0
dipole_dipole_energies = 0.0
strandC = dict.fromkeys([k for k in range(0,len(res_len))])
for key in strandC.keys():
    strandC[key] = {'anion':[],'cation':[],'charge_p':[],'charge_n':[]}
for i in range(0,len(res_len)):
#loop to go through each residue
    anion = []
    cation = []
    anion_of_ith_residue = []
    cation_of_ith_residue = []
    charge_n_of_ith_residue = []
    charge_p_of_ith_residue = []
    for k in range(0,len(residue_dict[res_len[i]]['x'])):
        #loop to go through each atom of ith residue
        x = residue_dict[res_len[i]]['x'][k]
        y = residue_dict[res_len[i]]['y'][k]
        z = residue_dict[res_len[i]]['z'][k]
        c = residue_dict[res_len[i]]['charge'][k]
        if (residue_dict[res_len[i]]['charge'][k] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
            anion_of_ith_residue.append([x,y,z])
            charge_n_of_ith_residue.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
            cation_of_ith_residue.append([x,y,z])
            charge_p_of_ith_residue.append(c)
    strandC[i]['anion'] = anion_of_ith_residue
    strandC[i]['cation'] = cation_of_ith_residue
    strandC[i]['charge_p'] = charge_p_of_ith_residue
    strandC[i]['charge_n'] = charge_n_of_ith_residue

##### For every residue starting from i=1 to n-1 calculate energies

In [29]:
 #if (residue_dict[res_len[i]]['charge'][k-1] < 0.0 ):
list_of_anions_for_calculations = []
list_of_cations_for_calculations = []
list_of_pcharge = []
list_of_ncharge = []
dipole_energy = 0.0
dipole_dipole_energies = 0.0
i = 1
while (i<len(res_len)-1):
# for i in range(1,len(res_len)-1):
    print("At residue "+str(i))
    #consider i-1,i,i+1 neighbouring residues for the calculations of dipoles and interactions.
    list_of_anions_for_calculations = strandC[i]['anion'] + strandC[i-1]['anion'] + strandC[i+1]['anion']
    list_of_cations_for_calculations = strandC[i]['cation'] + strandC[i-1]['cation'] + strandC[i+1]['cation']
    list_of_pcharge = strandC[i]['charge_p'] + strandC[i-1]['charge_p'] + strandC[i+1]['charge_p']
    list_of_ncharge = strandC[i]['charge_n'] + strandC[i-1]['charge_n'] + strandC[i+1]['charge_n']
    #Skip a residue in between to avoid adding up inter-residual interactiosn twice.
    i = i+2
    monopole_list = list_of_anions_for_calculations+list_of_cations_for_calculations
    monopole_charge_list = list_of_ncharge+list_of_pcharge
    monopole_energy = calculate_monopole_monopole_energies(monopole_list,monopole_charge_list)
    
    dipole_energy = dipole_energy + calculate_dipole_energies(list_of_anions_for_calculations,list_of_cations_for_calculations,
                              list_of_ncharge,list_of_pcharge)
    dipole_dipole_energies = dipole_dipole_energies + calculate_dipole_dipole_energies(list_of_anions_for_calculations,list_of_cations_for_calculations,list_of_ncharge,list_of_pcharge)

print(str(monopole_energy/1000.0)+" kjoule")
print(str(dipole_energy)+" kjoule")
print(str(dipole_dipole_energies/1000.0)+" kjoule")


At residue 1


KeyboardInterrupt: 

### -----------------strand D calculations------------------------

#### create atom_file [contains coordinates and their partial charges]

In [16]:
filepath = "AA_files/"
filename = "molecule_file_Chain_D.xlsx"
molecule_data_D = pd.read_excel(filepath+filename,engine="openpyxl")  

In [17]:
molecule_data_D[0:3]

,Atom_number,Residue_name,Chain_name,Atom_name,x,y,z,charge,mass
0,1,1C,D,O5',3.157,5.108,5.408,-0.66,15.9994
1,2,1C,D,H5T,3.137,5.030,5.351,0.43,1.0080
2,3,1C,D,C5',3.201,5.058,5.535,0.05,12.0110


#### create a dictionary containing every residue and its atoms coordinates and charges


In [18]:
residue_name_list_D = []
residue_dict_D = {}
for residue in molecule_data_D['Residue_name']:
    residue_name_list_D.append(residue)
residue_dict_D = dict.fromkeys(residue_name_list_D) 
for key in residue_dict_D.keys():
    residue_dict_D[key] = {'x':[],'y':[],'z':[],'charge':[],'mass':[]}

#### For every residue, get the atoms and their coords, mass and charges.


In [19]:
for item,row in molecule_data_D.iterrows():
    residue_dict_D[row['Residue_name']]['x'].append(row['x'])
    residue_dict_D[row['Residue_name']]['y'].append(row['y'])
    residue_dict_D[row['Residue_name']]['z'].append(row['z'])
    residue_dict_D[row['Residue_name']]['charge'].append(row['charge'])
    residue_dict_D[row['Residue_name']]['mass'].append(row['mass'])

#### for each residue get the list of anions and cations

In [20]:
#check partial charge of each atom in the residue and divide into cations and anions
residue_name_list2_D = []
[ residue_name_list2_D.append(x) for x in residue_name_list_D if x not in residue_name_list2_D  ]
cation = []
anion = []
charge_n = []
charge_p = []
res_len = residue_name_list2_D
dipole_energy = 0.0
dipole_dipole_energies = 0.0
strandD = dict.fromkeys([k for k in range(0,len(res_len))])
for key in strandD.keys():
    strandD[key] = {'anion':[],'cation':[],'charge_p':[],'charge_n':[]}
for i in range(0,len(res_len)):
#loop to go through each residue
    anion = []
    cation = []
    anion_of_ith_residue = []
    cation_of_ith_residue = []
    charge_n_of_ith_residue = []
    charge_p_of_ith_residue = []
    for k in range(0,len(residue_dict_D[res_len[i]]['x'])):
        #loop to go through each atom of ith residue
        x = residue_dict_D[res_len[i]]['x'][k]
        y = residue_dict_D[res_len[i]]['y'][k]
        z = residue_dict_D[res_len[i]]['z'][k]
        c = residue_dict_D[res_len[i]]['charge'][k]
        if (residue_dict_D[res_len[i]]['charge'][k] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
            anion_of_ith_residue.append([x,y,z])
            charge_n_of_ith_residue.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
            cation_of_ith_residue.append([x,y,z])
            charge_p_of_ith_residue.append(c)
    strandD[i]['anion'] = anion_of_ith_residue
    strandD[i]['cation'] = cation_of_ith_residue
    strandD[i]['charge_p'] = charge_p_of_ith_residue
    strandD[i]['charge_n'] = charge_n_of_ith_residue

##### For every residue starting from i=1 to n-1 calculate energies

In [26]:
 #if (residue_dict[res_len[i]]['charge'][k-1] < 0.0 :
list_of_anions_for_calculations = []
list_of_cations_for_calculations = []
list_of_pcharge = []
list_of_ncharge = []
dipole_energy = 0.0
dipole_dipole_energies = 0.0
i = 1
while (i<len(res_len)-1):
# for i in range(1,len(res_len)-1):
    print("At residue "+str(i))
    #consider i-1,i,i+1 neighbouring residues for the calculations of dipoles and interactions.
    list_of_anions_for_calculations = strandD[i]['anion'] + strandD[i-1]['anion'] + strandD[i+1]['anion']
    list_of_cations_for_calculations = strandD[i]['cation'] + strandD[i-1]['cation'] + strandD[i+1]['cation']
    list_of_pcharge = strandD[i]['charge_p'] + strandD[i-1]['charge_p'] + strandD[i+1]['charge_p']
    list_of_ncharge = strandD[i]['charge_n'] + strandD[i-1]['charge_n'] + strandD[i+1]['charge_n']
    #Skip a residue in between to avoid adding up inter-residual interactiosn twice.
    i = i+2
    monopole_list = list_of_anions_for_calculations+list_of_cations_for_calculations
    monopole_charge_list = list_of_ncharge+list_of_pcharge
    monopole_energy = calculate_monopole_monopole_energies(monopole_list,monopole_charge_list)
    dipole_energy = dipole_energy + calculate_dipole_energies(list_of_anions_for_calculations,list_of_cations_for_calculations,list_of_ncharge,list_of_pcharge)
    dipole_dipole_energies = dipole_dipole_energies + calculate_dipole_dipole_energies(list_of_anions_for_calculations,
                            list_of_cations_for_calculations,list_of_ncharge,list_of_pcharge)
print(str(monopole_energy/1000.0)+" kjoule")
print(str(dipole_energy/1000.0)+" kjoule")
print(str(dipole_dipole_energies/1000.0)+" kjoule")

At residue 1
At residue 3
At residue 5
At residue 7
At residue 9
At residue 11
At residue 13
At residue 15
At residue 17
At residue 19
-2.427627749289627e-29 kjoule
-1.5021174628551023e-27 kjoule
3.5209639443983643e-26 kjoule


#### Calculate energies of strand C and strand D together i.e. dsRNA

In [30]:
# to ensure that this works for unequal length strands as well
list_of_anions_for_calculations_strandC = []
list_of_cations_for_calculations_strandC = []
list_of_pcharge_strandC = []
list_of_ncharge_strandC = []

list_of_anions_for_calculations_strandD = []
list_of_cations_for_calculations_strandD = []
list_of_pcharge_strandD = []
list_of_ncharge_strandD = []
i = 1
dipole_energy = 0.0
dipole_dipole_energies = 0.0
while (i<len(strandC) and i<len(strandD)):
    
    #consider i-1,i,i+1 neighbouring residues for the calculations of dipoles and interactions.
    print("At residue "+str(i))
    
    list_of_anions_for_calculations_strandC = strandC[i]['anion'] + strandC[i-1]['anion'] + strandC[i+1]['anion']
    list_of_cations_for_calculations_strandC = strandC[i]['cation'] + strandC[i-1]['cation'] + strandC[i+1]['cation']
    list_of_pcharge_strandC = strandC[i]['charge_p'] + strandC[i-1]['charge_p'] + strandC[i+1]['charge_p']
    list_of_ncharge_strandC = strandC[i]['charge_n'] + strandC[i-1]['charge_n'] + strandC[i+1]['charge_n']

    list_of_anions_for_calculations_strandD = strandD[i]['anion'] + strandD[i-1]['anion'] + strandD[i+1]['anion']
    list_of_cations_for_calculations_strandD = strandD[i]['cation'] + strandD[i-1]['cation'] + strandD[i+1]['cation']
    list_of_pcharge_strandD = strandD[i]['charge_p'] + strandD[i-1]['charge_p'] + strandD[i+1]['charge_p']
    list_of_ncharge_strandD = strandD[i]['charge_n'] + strandD[i-1]['charge_n'] + strandD[i+1]['charge_n']
    #Skip a residue in between to avoid adding up inter-residual interactiosn twice.
    i = i+2
    
    monopole_list = list_of_anions_for_calculations_strandC + list_of_cations_for_calculations_strandC + list_of_anions_for_calculations_strandD + list_of_cations_for_calculations_strandD
    monopole_charge_list = list_of_ncharge_strandC + list_of_pcharge_strandC + list_of_ncharge_strandD + list_of_pcharge_strandD
    monopole_energy = calculate_monopole_monopole_energies(monopole_list,monopole_charge_list)
    
    dipole_energy = dipole_energy + calculate_dipole_energies(list_of_anions_for_calculations,list_of_cations_for_calculations,list_of_ncharge,list_of_pcharge)
    
    dipole_energy = dipole_energy + calculate_dipole_energies(list_of_anions_for_calculations_strandC,
                              list_of_cations_for_calculations_strandD,
                              list_of_ncharge_strandC, list_of_pcharge_strandD)
    
    dipole_energy = dipole_energy + calculate_dipole_energies(list_of_anions_for_calculations_strandD,
                              list_of_cations_for_calculations_strandC,
                              list_of_ncharge_strandD,list_of_pcharge_strandC)
    
    
    dipole_dipole_energies = dipole_dipole_energies +  calculate_dipole_dipole_energies(list_of_cations_for_calculations_strandC,
                            list_of_cations_for_calculations_strandD,list_of_pcharge_strandC,
                                     list_of_pcharge_strandD)
    
    dipole_dipole_energies = dipole_dipole_energies + calculate_dipole_dipole_energies(list_of_cations_for_calculations_strandC,
                        list_of_anions_for_calculations_strandD,list_of_pcharge_strandC,
                             list_of_ncharge_strandD)
    
    dipole_dipole_energies = dipole_dipole_energies + calculate_dipole_dipole_energies(list_of_anions_for_calculations_strandC,
                        list_of_cations_for_calculations_strandD,list_of_ncharge_strandC,
                             list_of_pcharge_strandD)
    
    dipole_dipole_energies = dipole_dipole_energies + calculate_dipole_dipole_energies(list_of_anions_for_calculations_strandC,
                        list_of_anions_for_calculations_strandD,list_of_ncharge_strandC,
                                 list_of_ncharge_strandD)
    
print(str(monopole_energy/1000.0)+" kjoule")
print(str(dipole_energy/1000.0)+" kjoule")
print(str(dipole_dipole_energies/1000.0)+" kjoule")

At residue 1
-4.191716849163659e-29 kjoule
-1.5944296221841736e-28 kjoule
1.994334132129284e-24 kjoule
At residue 3
-5.080693349733292e-29 kjoule
-3.402377375253918e-28 kjoule
4.7456735907822275e-24 kjoule
At residue 5
-4.8774510200532576e-29 kjoule
-5.43214200369889e-28 kjoule
7.286285102323859e-24 kjoule
At residue 7
-4.760123884497692e-29 kjoule
-7.611156208850905e-28 kjoule
9.757663452555831e-24 kjoule
At residue 9
-4.833038395033402e-29 kjoule
-1.0390575569109866e-27 kjoule
1.2255917576770058e-23 kjoule
At residue 11
-4.913110138990091e-29 kjoule
-1.2691942286872633e-27 kjoule
1.4767635319534847e-23 kjoule
At residue 13
-4.9340596414586885e-29 kjoule
-1.4502791043992536e-27 kjoule
1.7315218255855278e-23 kjoule
At residue 15
-4.66146207337941e-29 kjoule
-1.6147747507590175e-27 kjoule
1.9552862748014011e-23 kjoule
At residue 17
-5.142383602532626e-29 kjoule
-1.776642830256704e-27 kjoule
2.231086061204737e-23 kjoule
At residue 19
-4.8253300042825325e-29 kjoule
-1.9283661544577117e-27

#### -------------------------------------- Extra code ------------------------------------------------------------

In [ ]:
residue_name_list2 = []
[ residue_name_list2.append(x) for x in residue_name_list if x not in residue_name_list2  ]
cation = []
anion = []
charge_n = []
charge_p = []
res_len = residue_name_list2
dipole_energy = 0.0
dipole_dipole_energies = 0.0
strandA = dict.fromkeys([k for k in range(0,len(res_len))])
for key in strandA.keys():
    strandA[key] = {'anion':[],'cation':[],'charge_p':[],'charge_n':[]}
    #check partial charge of each atom in the residue 
    #and divide into cations and anions
for i in range(1,len(res_len)):
#loop to go through each residue
    anion = []
    cation = []
    anion_of_ith_residue = []
    cation_of_ith_residue = []
    charge_n_of_ith_residue = []
    charge_p_of_ith_residue = []
    for k in range(0,len(residue_dict[res_len[i]]['x'])):
        #loop to go through each atom of ith residue
        x = residue_dict[res_len[i]]['x'][k]
        y = residue_dict[res_len[i]]['y'][k]
        z = residue_dict[res_len[i]]['z'][k]
        c = residue_dict[res_len[i]]['charge'][k]
        if (residue_dict[res_len[i]]['charge'][k] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
            anion_of_ith_residue.append([x,y,z])
            charge_n_of_ith_residue.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
            cation_of_ith_residue.append([x,y,z])
            charge_p_of_ith_residue.append(c)
     for k_prev in range(0,len(residue_dict[res_len[i-1]]['x'])):
        x = residue_dict[res_len[i-1]]['x'][k]
        y = residue_dict[res_len[i-1]]['y'][k]
        z = residue_dict[res_len[i-1]]['z'][k]
        c = residue_dict[res_len[i-1]]['charge'][k]
        if (residue_dict[res_len[i-1]]['charge'][k] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
            anion_of_ith_residue.append([x,y,z])
            charge_n_of_ith_residue.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
            cation_of_ith_residue.append([x,y,z])
            charge_p_of_ith_residue.append(c)
     for k_prev in range(0,len(residue_dict[res_len[i+1]]['x'])):
        x = residue_dict[res_len[i-1]]['x'][k]
        y = residue_dict[res_len[i-1]]['y'][k]
        z = residue_dict[res_len[i-1]]['z'][k]
        c = residue_dict[res_len[i-1]]['charge'][k]
        if (residue_dict[res_len[i-1]]['charge'][k] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
            anion_of_ith_residue.append([x,y,z])
            charge_n_of_ith_residue.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
            cation_of_ith_residue.append([x,y,z])
            charge_p_of_ith_residue.append(c)
        
#         if (residue_dict[res_len[i]]['charge'][k-1] < 0.0 ):
        if (residue_dict[res_len[i-1]]['charge'][k-1] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
        if (residue_dict[res_len[i]]['charge'][k+1] < 0.0 ):
            anion.append([x,y,z])
            charge_n.append(c)
        else:
            cation.append([x,y,z])
            charge_p.append(c)
    dipole_energy = dipole_energy + calculate_dipole_energies(anion,cation,charge_n,charge_p)
    dipole_dipole_energies = dipole_dipole_energies + calculate_dipole_dipole_energies(anion,cation,charge_n,charge_p)
    strandA[i]['anion'] = anion_of_ith_residue
    strandA[i]['cation'] = cation_of_ith_residue
    strandA[i]['charge_p'] = charge_p_of_ith_residue
    strandA[i]['charge_n'] = charge_n_of_ith_residue
print(dipole_energy)
print(dipole_dipole_energies)